## Coroutine
- yield: Main-routine <-> Sub-routine
- coroutine 제어, coroutine 상태, 양방향 값 전송
- yield from

- sub-routine: 메인 루틴에서 return에 의해 호출 부분으로 돌아가 다시 프로세스
- coroutine: 루틴 실행 중 멈춤 가능. 특정 위치로 돌아갔다가 다시 원래 위치로 돌아와 수행 가능 -> 동시성 프로그래밍 가능! 하나의 쓰레드에서 실행하기 때문에 스케줄링 오버헤드 매우 적다.
- thread: 운영체제 단에서 생성. 파이썬은 기본적으로 싱글 쓰레드로 운영되며, 멀티쓰레드는 공유되는 자원의 교착 상태 발생 가능성이 있어 복잡하고 어렵다. 컨텍스트 스위칭 비용 발생. 자원 소비 증가.

In [2]:
def coroutine1():
    print('>>> coroutine started.')
    i = yield  # 양방향 통신 가능!
    print('>>> coroutine received : {}'.format(i))

### Generator 선언

In [8]:
c1 = coroutine1()
c1, type(c1)

(<generator object coroutine1 at 0x121a4e4f8>, generator)

#### 기본으로 None 전달

In [9]:
next(c1)

>>> coroutine started.


In [10]:
c1.send(100)

>>> coroutine received : 100


StopIteration: 

In [11]:
c2 = coroutine1()
c2.send(100)

TypeError: can't send non-None value to a just-started generator

>`yield`로 일단 멈춰놔야 `send`로 값을 전달할 수 있음

### 코루틴 예제
- GEN_CREATED: 처음 대기 상태
- GEN_RUNNING: 실행 상태
- GEN_SUSPENDED: yield 대기 상태
- GEN_CLOSED: 실행 완료 상태

In [12]:
def coroutine2(x):
    print('>>> coroutine started: {}'.format(x))
    y = yield x
    print('>>> coroutine received: {}'.format(y))
    z = yield x + y
    print('>>> coroutine received: {}'.format(z))
    
c3 = coroutine2(10)

In [13]:
from inspect import getgeneratorstate

`next` method 호출 전

In [14]:
getgeneratorstate(c3)

'GEN_CREATED'

In [15]:
next(c3)

>>> coroutine started: 10


10

y값 받기 전 대기 상태 

In [16]:
getgeneratorstate(c3)

'GEN_SUSPENDED'

In [17]:
c3.send(15)  # return 10 + 15

>>> coroutine received: 15


25

In [19]:
c3.send(20)

>>> coroutine received: 20


StopIteration: 

실행 완료 상태

In [20]:
getgeneratorstate(c3)

'GEN_CLOSED'

### decorator 패턴
- 선 next 호출 없이 coroutine 생성 가능

In [23]:
from functools import wraps

def coroutine(func):
    '''Decorator run until yield'''
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [24]:
@coroutine
def sumer():
    total = 0
    term = 0
    while True:
        term = yield total
        total += term

In [29]:
su = sumer()

print(su.send(1))
print(su.send(10))
print(su.send(100))
print(su.send(1000))
print(su.send(10000))

1
11
111
1111
11111


### 예외 처리 패턴

In [30]:
class SampleException(Exception):
    '''설명에 사용할 예외 유형'''
    
def coroutine_except():
    print('>>> coroutine started.')
    try:
        while True:
            try:
                x = yield
            except SampleException:
                print('-> SampleException handled. Continuing..')
            else:
                print('-> coroutine received:', x)
    finally:
        print('-> coroutine ending')

In [31]:
exe_co = coroutine_except()

next(exe_co)

>>> coroutine started.


In [32]:
exe_co.send(10)

-> coroutine received: 10


In [33]:
exe_co.send(100)

-> coroutine received: 100


In [34]:
exe_co.throw(SampleException)

-> SampleException handled. Continuing..


In [35]:
exe_co.send(1000)

-> coroutine received: 1000


In [36]:
exe_co.close()  # GEN_CLOSED

-> coroutine ending


In [37]:
exe_co.send(10)

StopIteration: 

>coroutine 종료 후 보내므로 `StopIteration` 발생

### return 패턴

In [43]:
def averager_re():
    total = 0.0
    cnt = 0
    avg = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        cnt += 1
        avg = total / cnt
    return 'Average: {}'.format(avg)

In [44]:
avg_co = averager_re()

In [45]:
next(avg_co)

In [46]:
avg_co.send(10)
avg_co.send(20)
avg_co.send(30)

`None`을 보내 종료를 시켜야 결과값을 확인할 수 있는 상태

In [47]:
try:
    avg_co.send(None)
except StopIteration as e:
    print(e.value)

Average: 20.0


>`StopIteration`의 e.value 값으로 확인 가능

### `yield from` 패턴
- `StopIteration` 자동 처리 (3.7버전부터 yield from -> await로 변경)
- 중첩 코루틴 처리

In [48]:
def gen1():
    for x in 'AB':
        yield x
    for y in range(1, 4):
        yield y

In [49]:
t1 = gen1()

next(t1)

'A'

In [50]:
next(t1)

'B'

In [51]:
next(t1)

1

In [52]:
next(t1)

2

In [53]:
next(t1)

3

In [54]:
next(t1)

StopIteration: 

In [55]:
t2 = gen1()

In [56]:
list(t2)

['A', 'B', 1, 2, 3]

>값을 전부 반환하여 리스트 값으로 만들어줌

위와 같은 중첩 coroutine은 `yield from`으로 편하게 치환할 수 있음

In [57]:
def gen2():
    yield from 'AB'
    yield from range(1, 4)

In [58]:
t3 = gen2()

next(t3)

'A'

In [59]:
next(t3)

'B'

In [60]:
next(t3)

1

In [61]:
next(t3)

2

In [62]:
next(t3)

3

In [63]:
next(t3)

StopIteration: 

In [65]:
t4 = gen2()

list(t4)

['A', 'B', 1, 2, 3]

In [66]:
def gen3_sub():
    print('Sub coroutine.')
    x = yield 10
    print('Recv:', str(x))
    x = yield 100
    print('Recv:', str(x))
    
def gen4_main():
    yield from gen3_sub()

In [67]:
t5 = gen4_main()

next(t5)

Sub coroutine.


10

In [68]:
t5.send(7)

Recv: 7


100

In [69]:
t5.send(77)

Recv: 77


StopIteration: 